# Project 3: Beating the weather

**It is an explicit requirement that you should <u>not</u> be using machine learning in the project element of this course.**

<a href="https://open-meteo.com/">Open-Meteo</a> provides free access to historic weather data globally and also forecasts into the future. 


Example scenarios:

1)	You are approached by the owner of a hot air balloon company. They are looking to expand by adding a new location for hot air balloon flights. Hot air balloon flights can only take place when the wind is below 7mph and the weather is fine (ie sunny or cloudy but very little chance of rain). It needs to be in the UK but it also needs to be somewhere beautiful, perhaps a national park or area of outstanding natural beauty. If they can give you a list of places can you work out where, based on historical data is the best place (ie most number of days they will be able to fly per year). Perhaps you could even help them visualise number of estimated days flying on a contour map of the UK. By the way his brother owns a windsurfing business where strong winds are good and weak winds are bad.

2)	A number of councils have been looking at implementing congestion charges in their cities. To do so they will need to make a case that the air quality in their city due to traffic is too poor relative to the countryside. From this they will be able to argue that congestion charges are necessary on the basis of protecting people's health. Regardless of whether you believe it is about health or making money can you supply them with a tool that allows them to investigate the air quality in different places of interest. 

3)	Looking at the underlying causes of changes in the weather has been constantly in the news due to the emphasis on climate change. The Open-Meteo weather api has 80 years of historical data. However, there are other interesting patterns you could look into. Sunspots are small dark regions of the Sun. There is a known cycle ~12 years in the number of visible sunspots. You can <a href="https://www.sidc.be/SILSO/datafiles">download csv datafiles of the number of sunspots</a>. Does this have any effect on our Weather?

You can use / adapt one of these or come up with something else. I look forward to seeing what you come up with.

A Conda environment file project.yml is included to get you up and running.

In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
import matplotlib.pyplot as plt

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

You can explore the data accessible through their api by looking at the <a href="https://open-meteo.com/en/docs">documentation page</a>. The Open-Meteo API enables you to make requests using python that will return data according to the parameters you set. You are limited to 10000 API calls per day so don't go crazy or they'll likely block your computer. If you want to repeatedly test parts of your code then make an API call and store the data locally in a file. For full details on making a request, read the <a href="https://pypi.org/project/openmeteo-requests/">python api documentation</a>

In [ ]:
"""Some setup for the API call
"""

#Location for the weather data Place or 'Latitude,Longitude'
Location=('52.9540','-1.1550') # 'Nottingham,UK'

#Format YYYY-MM-DD
StartDate = "2024-06-09"
EndDate= "2024-06-23"

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": Location[0],
	"longitude": Location[1],
	"start_date": StartDate,
	"end_date": EndDate,
	"hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

In [ ]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(hourly_data)

In [ ]:
thin_xticks_factor=100
plt.plot(hourly_dataframe['date'], hourly_dataframe['temperature_2m'])
plt.xticks(hourly_dataframe['date'][::thin_xticks_factor])
plt.ylabel('Temperature (C)')
plt.xlabel('Day')